<a href="https://colab.research.google.com/github/shriman/storelift_sales_prediction/blob/main/storelift_model_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# test runtime config
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


# Imports

In [13]:
import os
import os
import seaborn as sns
from os import path
import pandas as pd
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_columns = None
pd.options.display.max_colwidth = 50

data_path = '/content/drive/My Drive/Colab Notebooks/storelift/data'
output_path = '/content/drive/My Drive/Colab Notebooks/storelift/outputs'

# Some helper functions

In [14]:
def get_source_filename(data_path, filename, suffix=None):
    if suffix:
        return os.path.join(data_path, filename + '.' + suffix)
    
    return os.path.join(data_path, filename + '.csv' )

# Read dataset

In [15]:
df_full_filename = get_source_filename(output_path, 'df_full')
df_full = pd.read_csv(df_full_filename)

In [16]:
df_full.head()

,day,product_id,day_product_id,target,price_avg_daily,has_expiration_int,category_combined_code,product_id_code,full_name_len,full_name_word_0,full_name_word_1,full_name_word_2,full_name_word_3,full_name_word_4,full_name_word_5,full_name_word_6,full_name_word_7,full_name_word_8,full_name_word_9,full_name_word_10,full_name_word_11,full_name_word_12,full_name_word_13,full_name_word_14,full_name_word_15,category_combined_len,category_combined_word_0,category_combined_word_1,levenshtein_dist,levenshtein_dist_ratio,returned_count,promotion_count,month,weekday,vat,is_confinement,is_summer,is_weekend
0,1,5d0794866885630014681b97,1_5d0794866885630014681b97,8.0,268.03336,0.0,23.0,29.0,8.0,158.0,75.0,582.0,381.0,419.0,10.0,75.0,61.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,13.0,41.0,0.233766,0.0,0.0,8,5,0.2,0.0,1.0,1.0
1,2,5d0794866885630014681b97,2_5d0794866885630014681b97,8.0,268.03336,0.0,23.0,29.0,8.0,158.0,75.0,582.0,381.0,419.0,10.0,75.0,61.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,13.0,41.0,0.233766,0.0,0.0,8,5,0.2,0.0,1.0,1.0
2,3,5d0794866885630014681b97,3_5d0794866885630014681b97,8.0,268.03336,0.0,23.0,29.0,8.0,158.0,75.0,582.0,381.0,419.0,10.0,75.0,61.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,13.0,41.0,0.233766,0.0,0.0,8,5,0.2,0.0,1.0,1.0
3,4,5d0794866885630014681b97,4_5d0794866885630014681b97,8.0,268.03336,0.0,23.0,29.0,8.0,158.0,75.0,582.0,381.0,419.0,10.0,75.0,61.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,13.0,41.0,0.233766,0.0,0.0,8,5,0.2,0.0,1.0,1.0
4,5,5d0794866885630014681b97,5_5d0794866885630014681b97,8.0,268.03336,0.0,23.0,29.0,8.0,158.0,75.0,582.0,381.0,419.0,10.0,75.0,61.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,3.0,13.0,41.0,0.233766,0.0,0.0,8,5,0.2,0.0,1.0,1.0


In [17]:
# divide sets
df_test = df_full[df_full['month'] == 2]
df_valid = df_full[df_full['month'] == 1]
df_train = df_full[df_full['month'] > 2 ]
print(df_test.shape)
print(df_valid.shape)
print(df_train.shape)
columns_to_drop = ['product_id', 'day_product_id', 'target']

(32704, 38)
(9273, 38)
(52046, 38)


# ML Model: random forest regressor

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Initialise the random forest model 
RandForest = RandomForestRegressor(n_jobs= -1, random_state = 0, bootstrap=True)

# Set paramters for Grid Search
param_grid =  {'n_estimators':[200, 300, 400, 500, 600],
               'max_features':[0.1, 0.3, 0.6]
              }

# Initialise Gridsearch CV with 5 fold corssvalidation and neggative root_mean_squared_error
Tuned_RandForest = GridSearchCV(estimator=RandForest, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5)

In [22]:
# Fit model & Time the process for training the model
import time
start_time = time.process_time()
Tuned_RandForest.fit(df_train.drop(columns=columns_to_drop), df_train['target'])
# End of fit time
print(time.process_time() - start_time, "Seconds")

223.55414378999998 Seconds


In [23]:
# Record the results for all models in a pandas dataframe and keep only the best model
Results = pd.DataFrame(Tuned_RandForest.cv_results_)
Results_Best = Results.loc[Results.rank_test_score==1]

In [24]:
print('Random Forest Regressor')
Results_Best

Random Forest Regressor


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,19.154623,0.266879,0.307191,0.002115,0.3,300,"{'max_features': 0.3, 'n_estimators': 300}",-5.98002,-2.994016,-3.954781,-3.710317,-2.227665,-3.77336,1.257202,1


In [28]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(df_train['target'], Tuned_RandForest.predict(df_train.drop(columns=columns_to_drop)), squared=False)
print("The root mean squared error (RMSE) on train set: {:.4f}".format(rmse))

The root mean squared error (RMSE) on train set: 0.2342


In [29]:
rmse = mean_squared_error(df_valid['target'], Tuned_RandForest.predict(df_valid.drop(columns=columns_to_drop)), squared=False)
print("The root mean squared error (RMSE) on validation set: {:.4f}".format(rmse))

The root mean squared error (RMSE) on validation set: 2.2330
